In [118]:
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2
log_problem = pd.read_csv('Log_Problem.csv')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Refer to this article: https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python

In [110]:
import networkx as nx
from networkx.algorithms import community #This part of networkx, for community detection, needs to be imported separately.
from operator import itemgetter

In [3]:
node_list = log_problem['ucid'].unique()

In [4]:
log_problem.head()

,timestamp_TW,uuid,ucid,upid,problem_number,exercise_problem_repeat_session,is_correct,total_sec_taken,total_attempt_cnt,used_hint_cnt,is_hint_used,is_downgrade,is_upgrade,level
0,2019-05-26 21:00:00 UTC,FLy+lviglNR5Y1l0Xiijnl6QHySBcpKHJLCtQ6ogm2Q=,KDOmuTrY/IJzDP4kIgIYCBiGyTymsJ8Iy4cDB35WGYg=,Vbs92l4JmdiWkUEm/iahxnUTaac2oN1IlUtXB7JcfoE=,18,2,True,33,1,0,False,False,True,3
1,2019-05-17 16:30:00 UTC,+Gqj2nalc6M9fusyVECTC0AN7UQdDQTXESIuElkDltU=,COZ39Wo+uIUO2s7c2VGEHjJf6Vx0xifxVAiaeHtaTdk=,Ek+pIeHNNoEo0tGEq91eBcBmGgy3+A5RWhpj95zTyHM=,4,1,True,8,1,0,False,NaN,NaN,0
2,2019-05-15 19:15:00 UTC,6D5QN8j8ng/VR74ES3A0zqAj0bIFFyaKjKEj8ZyXjQ8=,TwyqyV1uJYlDAX8wX/PtTCVZEBo/APIVfTzzleGkNCQ=,1MBa2f5Qog4JBoAuUfJf0fxeJctdEirAqKgfsg246eI=,9,1,True,17,1,0,False,NaN,NaN,0
3,2019-05-05 14:45:00 UTC,GgTZuCqZXObthtK6GAwqvlHrTMm5pKHWeezQxL/pcKc=,tBo6ECyT8IlKAM8UhQHWkqv92PRLcSiwuerfC7vNX+w=,kdMy2nG+QVMjPkuaMEWs0yV/sYZVoG1vm7zM0fCy+qk=,2,1,True,10,1,0,False,NaN,NaN,0
4,2019-05-14 16:45:00 UTC,JMNKWoU0CkMSzgQ8bCnmCYlD8jEzAVge3lHMYLXKM2g=,vVpSKAMQbTMvtdERR0ksOeRmmaFt0R210t4Z//0RpPA=,jjPR8fmkLSFoCQQYB4g6kI8mgdcK3sKtMirKUvfmZIk=,6,1,True,98,1,0,False,NaN,NaN,0


In [20]:
log_problem.sort_values(by=['timestamp_TW'], inplace=True)

In [75]:
user_to_content = log_problem.groupby(['uuid'], sort=False)['ucid'].apply(lambda x: x.tolist()).reset_index()

In [76]:
user_to_content['to'] = user_to_content['ucid'].apply(lambda x: x[1:])
user_to_content['from'] = user_to_content['ucid'].apply(lambda x: x[:-1])

### Look at the content mapping of a particular user

In [77]:
log_problem[log_problem['uuid'] == 'U+lqK/FKWkPuoNUM1AbGyrKZfXeQrRRoKOToKrjqDt4='].head()

,timestamp_TW,uuid,ucid,upid,problem_number,exercise_problem_repeat_session,is_correct,total_sec_taken,total_attempt_cnt,used_hint_cnt,is_hint_used,is_downgrade,is_upgrade,level
105021,2018-08-01 07:45:00 UTC,U+lqK/FKWkPuoNUM1AbGyrKZfXeQrRRoKOToKrjqDt4=,CPI+5YCeEmhqdk6znJeii6jJUNl1QWGEvwCUJ6uLflg=,9Ksy3bdm0HTi0D+cdKSBKDQJjo5GNahep91FqHUrpts=,2,1,True,8,1,0,False,NaN,NaN,0
11669798,2018-08-01 07:45:00 UTC,U+lqK/FKWkPuoNUM1AbGyrKZfXeQrRRoKOToKrjqDt4=,CPI+5YCeEmhqdk6znJeii6jJUNl1QWGEvwCUJ6uLflg=,/Wgjdl2BsldHZDdXXvzwGimusaMX548lqV2b7PgwXAs=,1,1,True,10,1,0,False,NaN,NaN,0
3768239,2018-08-01 07:45:00 UTC,U+lqK/FKWkPuoNUM1AbGyrKZfXeQrRRoKOToKrjqDt4=,CPI+5YCeEmhqdk6znJeii6jJUNl1QWGEvwCUJ6uLflg=,vsGlLPd9C58B8myBoGMGre2pDHjh62eRjsqX57D98fU=,5,1,True,6,1,0,False,False,True,1
10163558,2018-08-01 07:45:00 UTC,U+lqK/FKWkPuoNUM1AbGyrKZfXeQrRRoKOToKrjqDt4=,hv7kHCAIdj7thZUmlqz553leG5bFNYgzXmLfB5m4Xvw=,h3CI/U4QJd6mjYE5xRH8QEst8lRG7otYIz+q1V6Och4=,5,1,True,5,1,0,False,False,True,1
8372688,2018-08-01 07:45:00 UTC,U+lqK/FKWkPuoNUM1AbGyrKZfXeQrRRoKOToKrjqDt4=,CPI+5YCeEmhqdk6znJeii6jJUNl1QWGEvwCUJ6uLflg=,OSuDd4rDo2muXXwwJRU2DQVHkk6/JOGgNzfzNi4PMJM=,3,1,True,4,1,0,False,NaN,NaN,0


#### We can see that a user can do multiple questions consecutively(and hence multiple instances of the same 'ucid' is updated consecutively in the log). Thus i just drop rows when the course content is the same consecutively. 
- This is done in when creating data for networkx

In [78]:
# obtain a list of content path with no consecutive same content
user_to_content['unique_ucid'] = user_to_content['ucid'].apply(lambda x: [content for index, content in enumerate(x[:-1]) if x[index]!=x[index+1]])

In [79]:
# Test that for this user there is no repeated content
tc1_uuid = 'U+lqK/FKWkPuoNUM1AbGyrKZfXeQrRRoKOToKrjqDt4='
tc_d = user_to_content[user_to_content['uuid']==tc1_uuid]
assert len(tc_d['unique_ucid']) == len(np.unique(tc_d['uuid']))

# user that only did one content will have empty list of contents
tc2_uuid = 'Oj3eQxD25uE+DeBkWLSlwUQSYyFms0cBF6zGvMDpDIo=' # row 72753
tc_d = user_to_content[user_to_content['uuid']==tc2_uuid]
assert len(np.unique(tc_d['ucid'].values.tolist()[0]))==1

In [80]:
# Drop those rows with empty list
user_to_content['to_keep'] = user_to_content['unique_ucid'].apply(lambda x: np.nan if len(x)==0 else 1)
user_to_content = user_to_content.dropna(subset=['to_keep'])

# Create new source node list and the corresponding target list without any repeated consecutive contents
user_to_content.loc[:,'to2'] = user_to_content['unique_ucid'].apply(lambda x: x[1:])
user_to_content.loc[:,'from2'] = user_to_content['unique_ucid'].apply(lambda x: x[:-1])
user_to_content['to2']

In [ ]:
> Define the edge weights for the exercise?
    - Do we have the information to compute the weihgt such that we can compare paths against one another
    for src in range(num_exercise):
        for tgt in range(num_problems):
        - (Frequency of student for src to tgt) * (accuracy of student obtained by student from src to tgt)
        
> Define the edge weights for the problem?
    - Do length of the problem sequence
    for i in range(num_problems):
        - (Frequency of student for problem i) * (accuracy of student for problem i)
        


In [ ]:
def load(fname):
    G = nx.DiGraph()
    d = simplejson.load(open(fname))
    for item in d:
        for attribute, value in item.iteritems():
            subject_id, object_id = value['subject_id'], value['object_id']
            if G.has_edge(subject_id, object_id):
                # we added this one before, just increase the weight by one
                G[subject_id][object_id]['weight'] += 1 is_correct(src) -> is_correct(tgt)
            else:
                # new edge. add with weight=1
                G.add_edge(subject_id, object_id, weight=1)
    return G



### Create Directed Graphs

In [90]:
G = nx.DiGraph()
for index, row in user_to_content.iterrows():
    user = row['uuid']
    from_list = row['from2']
    to_list = row['to2']
    # always check that the number 
    assert len(from_list) == len(to_list)
    for src, tgt in zip(from_list, to_list):
        if G.has_edge(src, tgt):
            G[src][tgt]['weight'] += 1
        else:
            G.add_edge(src, tgt, weight=1)

### Create unDirected Graphs

In [112]:
G = nx.Graph()
for index, row in user_to_content.sample(10000).iterrows():
    user = row['uuid']
    from_list = row['from2']
    to_list = row['to2']
    # always check that the number 
    assert len(from_list) == len(to_list)
    for src, tgt in zip(from_list, to_list):
        if G.has_edge(src, tgt):
            G[src][tgt]['weight'] += 1
        else:
            G.add_edge(src, tgt, weight=1)

### For the sake of time, i sampled 10000 lol

In [113]:
user_to_content

,uuid,ucid,to,from,unique_ucid,to_keep,to2,from2
0,U+lqK/FKWkPuoNUM1AbGyrKZfXeQrRRoKOToKrjqDt4=,"[CPI+5YCeEmhqdk6znJeii6jJUNl1QWGEvwCUJ6uLflg=,...","[CPI+5YCeEmhqdk6znJeii6jJUNl1QWGEvwCUJ6uLflg=,...","[CPI+5YCeEmhqdk6znJeii6jJUNl1QWGEvwCUJ6uLflg=,...","[CPI+5YCeEmhqdk6znJeii6jJUNl1QWGEvwCUJ6uLflg=,...",1.0,"[hv7kHCAIdj7thZUmlqz553leG5bFNYgzXmLfB5m4Xvw=,...","[CPI+5YCeEmhqdk6znJeii6jJUNl1QWGEvwCUJ6uLflg=,..."
1,ZtYcfQTg++4WtnOkIdgI7O4odEyGxOP4ZchUy1ENNIY=,"[KpJfQutqPUoUbwWZ5oNFKeVhvKqXZcqL69vsPEHg9jY=,...","[KpJfQutqPUoUbwWZ5oNFKeVhvKqXZcqL69vsPEHg9jY=,...","[KpJfQutqPUoUbwWZ5oNFKeVhvKqXZcqL69vsPEHg9jY=,...","[KpJfQutqPUoUbwWZ5oNFKeVhvKqXZcqL69vsPEHg9jY=,...",1.0,"[suDCkQs0H6kriJ78abNYmrtyOrBj80FflKWAiW3yfD8=,...","[KpJfQutqPUoUbwWZ5oNFKeVhvKqXZcqL69vsPEHg9jY=,..."
2,j/Prqubd0XhHmteIdpCvOcq7IgV4Ib9qLXF1gwNI5ww=,"[h327f3+M2F6lvbXTOgg35VC4pac6YD68c7hjS2DPPiA=,...","[h327f3+M2F6lvbXTOgg35VC4pac6YD68c7hjS2DPPiA=,...","[h327f3+M2F6lvbXTOgg35VC4pac6YD68c7hjS2DPPiA=,...","[h327f3+M2F6lvbXTOgg35VC4pac6YD68c7hjS2DPPiA=,...",1.0,"[5gAwpmADo34wxzAw/auRsRgPsj+RvWPNWhOj3SdP0Ss=,...","[h327f3+M2F6lvbXTOgg35VC4pac6YD68c7hjS2DPPiA=,..."
3,tj/HawoHmjMZJFe3o2B5zeASGbnAuNUtzNd1YrtcHdw=,"[TKDAjftSYyI2PNmAkXdInD39P5BcPNa6a+qx0fxm4RA=,...","[TKDAjftSYyI2PNmAkXdInD39P5BcPNa6a+qx0fxm4RA=,...","[TKDAjftSYyI2PNmAkXdInD39P5BcPNa6a+qx0fxm4RA=,...","[TKDAjftSYyI2PNmAkXdInD39P5BcPNa6a+qx0fxm4RA=,...",1.0,"[NoskBq4Br1/gsHaS0Pkuhdx67DlKfRiCcj2jMAWpRS8=,...","[TKDAjftSYyI2PNmAkXdInD39P5BcPNa6a+qx0fxm4RA=,..."
4,AHtLCC16j7YXj6gCouJXMcLe7UapiMEO1DBn5xUfM6Q=,"[PGbCr6wCmzlyPoKCBQv8aOM0NW8HVk4PW+IU4iR6GRI=,...","[4p305X/NqL1Zx9j6BDJEpiQx090kFF3o/LTTGJt5s/E=,...","[PGbCr6wCmzlyPoKCBQv8aOM0NW8HVk4PW+IU4iR6GRI=,...","[PGbCr6wCmzlyPoKCBQv8aOM0NW8HVk4PW+IU4iR6GRI=,...",1.0,"[4p305X/NqL1Zx9j6BDJEpiQx090kFF3o/LTTGJt5s/E=,...","[PGbCr6wCmzlyPoKCBQv8aOM0NW8HVk4PW+IU4iR6GRI=,..."
...,...,...,...,...,...,...,...,...
72749,PiqckXsI/J2wiFgc6UkrSkHi0ZVx/+oHrTaFExnc0us=,"[jvSzlPLRjqqstMszjfw2Y1+0IOmVTo8P8l/rpJzPgNU=,...","[jvSzlPLRjqqstMszjfw2Y1+0IOmVTo8P8l/rpJzPgNU=,...","[jvSzlPLRjqqstMszjfw2Y1+0IOmVTo8P8l/rpJzPgNU=,...","[jvSzlPLRjqqstMszjfw2Y1+0IOmVTo8P8l/rpJzPgNU=,...",1.0,"[aQys6A/5FjGwd7dQ0KQ4h/5zmPrM7lbkwj2ZkuhMepo=,...","[jvSzlPLRjqqstMszjfw2Y1+0IOmVTo8P8l/rpJzPgNU=,..."
72751,yzzVBDVLH6D6WYl+lcybce8c0KIObCsssioFEXn2EzQ=,"[cvnGl+iUiKphfjbPcE0G7/HRE/Q51L3w0x0WhVLy1cs=,...","[5gAwpmADo34wxzAw/auRsRgPsj+RvWPNWhOj3SdP0Ss=,...","[cvnGl+iUiKphfjbPcE0G7/HRE/Q51L3w0x0WhVLy1cs=,...","[cvnGl+iUiKphfjbPcE0G7/HRE/Q51L3w0x0WhVLy1cs=,...",1.0,"[5gAwpmADo34wxzAw/auRsRgPsj+RvWPNWhOj3SdP0Ss=,...","[cvnGl+iUiKphfjbPcE0G7/HRE/Q51L3w0x0WhVLy1cs=,..."
72752,saB6CvIM32G0+RDry4wCo5ro6A1kI3fKYaKsjwQc+T4=,"[P9MBH2y9lTr+ueuFDr6VDrx3GSS1/MvPLRyklqdSyVM=,...","[P9MBH2y9lTr+ueuFDr6VDrx3GSS1/MvPLRyklqdSyVM=,...","[P9MBH2y9lTr+ueuFDr6VDrx3GSS1/MvPLRyklqdSyVM=,...","[P9MBH2y9lTr+ueuFDr6VDrx3GSS1/MvPLRyklqdSyVM=,...",1.0,[wZCkmlfJ0G2xyiucQ85Ms+DMGXRSg8yhQ+MiwtSHS30=],[P9MBH2y9lTr+ueuFDr6VDrx3GSS1/MvPLRyklqdSyVM=]
72755,90OCKSK8bkx4M+KdLllF8XNzykbJldJtxWaxQRuxPWY=,"[XPjtEuV2qgexNe8re/JecZlBLnzABLhXY2kd8BFHlRg=,...","[OnztoLe/KRAGCzT1fhys2SSvoBd4VIDTcFpP5fMpU68=,...","[XPjtEuV2qgexNe8re/JecZlBLnzABLhXY2kd8BFHlRg=,...","[XPjtEuV2qgexNe8re/JecZlBLnzABLhXY2kd8BFHlRg=,...",1.0,"[OnztoLe/KRAGCzT1fhys2SSvoBd4VIDTcFpP5fMpU68=,...","[XPjtEuV2qgexNe8re/JecZlBLnzABLhXY2kd8BFHlRg=,..."


#### 20k 

### Density

In [114]:
# DENSITY
# Density is the ratio of the actual edges in the network to all possible edges in the network
# Interpretation: 0 to 1, the bigger the more dense your network. 1 is a perfectly connect network
# This mean that 
density = nx.density(G)
print("Network density:", density)

Network density: 0.1152747659295939


### SSSP
######  SSSP

In [115]:
src =list(G.nodes)[0]
tgt =list(G.nodes)[-1]

>ew: 1.accuracy? 

In [116]:
# SSSP
# Trim the edge weights first before we try to run SSSP?
# Given some domain knowledge, eg what course the student wants to take, run the SSSP algorithm
# Some thing to consider is that if an intermediate node has a high degree centrality(more on that later),
# The SSSP may take very long since several shortest paths run through him as the mediator
# (This provide us with some hint that DSTA should focus on nodes with the highest degree centrality, 
# I.E make the courses better since these are important courses to take!)
path_length = nx.shortest_path(G, source=src, target=tgt)

print(f"Shortest path:", path_length)
print("Length of that path:", len(path_length)-1)

Shortest path: ['svTmVFY7kPYUWAdDc4l5Zaim90TY7RGkFLs/eaU4FVc=', 'DnZ+4F0wfZyawol4t+/bAvIm5PTPYUC/NSZakKr8g4I=', 'bGhEHzDDpgeN/wftiCCViepN+b6lTXEnxj1/8DxWeZ4=', 'RLVg2oNL54gAjL5WK5vK9JDSj1xUds0tDwVqAAfDTxg=']
Length of that path: 3


### Components and their Diameters

In [117]:
# COMPONENTS, AND COMPONENTS AND THIER DIAMETERS
# LOOK AT THE NUMBER OF COMPONENTS
# If we have more than one components, then this means that some course contents are very different from the 
# Eg interpretation: There is a learning path frokm length 8 between the two furthest apart courses in the network.

# If your Graph has more than one component, this will return False:
# print(nx.is_strongly_connected(G))
print(nx.is_connected(G))


# Next, use nx.connected_components to get the list of components,
# then use the max() command to find the largest one:
components = nx.connected_components(G)
largest_component = max(components, key=len)

# Create a "subgraph" of just the largest component
# Then calculate the diameter of the subgraph, just like you did with density.
# This ensures that nx.diameter will not throw an error
subgraph = G.subgraph(largest_component)
diameter = nx.diameter(subgraph)
print("Network diameter of largest component:", diameter)

True
Network diameter of largest component: 4


### Triadic Closure 

In [ ]:
# TRANSITIVITY
# Clustering coefficient is one way to measure Triadic closure as it measures the clustering tendency.
# Transitivity is the ratio of all triangles over all possible triangles
triadic_closure = nx.transitivity(G)
print("Triadic closure:", triadic_closure)

### Centrality Measures

#### degree

cluster 1: node A: metric(A) = 2, node b: 2. node c: 8
cluster 2: node A: 1, node b: 9, node c: 5
cluster 3: node A: 1, node b: 3, node c: 9

In [ ]:
## DEGREE
# Course with the highest degree are those most connected to other courses. These nodes are call Hubs.
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree') # store all the degrees in the node
### Find the top few nodes with the highest degree
#  Eg print(G.nodes['William Penn'])
# sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)
# print("Top 20 nodes by degree:")
# for d in sorted_degree[:20]:
#     print(d)

#### Betweenness and eigenvector Centrality

In [ ]:
## EIGENVECTOR CENTRALITY
# Eigenvector centrality useful for understanding which nodes can get information to other nodes quickly.
# Example: if you know a lot of well-connected people, you can spread a message very efficiently.
# So these are courses that are related to many of the important courses, and should be placed into the learning path?


## BETWEENNESS CENTRALITY
# Betweenness centrality looks at all the shortest paths that pass through a particular node. (Thus takes very long to calculate)
# Tells us which nodes are important not because they have lots of connections themselves, but because they stand between groups, 
# Giving the network connectivity and cohesion

betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality
eigenvector_dict = nx.eigenvector_centrality(G) # Run eigenvector centrality

# Assign each to an attribute in your network
nx.set_node_attributes(G, betweenness_dict, 'betweenness')
nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')

sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)

print("Top 20 nodes by betweenness centrality:")
for b in sorted_betweenness[:20]:
    print(b)
    
### You can see that nodes that have high degree also have high betweenness centrality
# What if you want to know learning path have high betweenness centrality and low degree?
# I.e which high betweenness nodes are unexpected?
## If we have high betweenness centrality and low degree, these could mean that the nodes are important nodes connecting
## distant parts of the graph.

#First get the top 20 nodes by betweenness as a list
top_betweenness = sorted_betweenness[:20]

#Then find and print their degree
for tb in top_betweenness: # Loop through top_betweenness
    degree = degree_dict[tb[0]] # Use degree_dict to access a node's degree, see footnote 2
    print("Name:", tb[0], "| Betweenness Centrality:", tb[1], "| Degree:", degree)

### Communities

In [ ]:
# COMMUNITIES
## Community detection with modularity
## Modularity is a measure of relative density in your network. 
## A community has a high density relative to other nodes within its module, but low density with those outside
## Modularity gives you an overall score of how fractious your network is, and that is a score
# that can be used to partition the network and return the inidividual communities

# Very dense networks are often more difficult to split into sensible partitions.
# Luckily, networks are not all that dense. Our network is only 0.223
# Although there are some built in approahces like 'minimum cut', modularity is not included in networkX
# We just use an addiitonal python module, YAY
communities = community.greedy_modularity_communities(G)
## the method greedy_modularity_communities() tries to determine the number of communities appropriate for the graphs
## Group all nodes into the subsets based on these communities.

modularity_dict = {} # Create a blank dictionary
for i,c in enumerate(communities): # Loop through the list of communities, keeping track of the number for the community
    for name in c: # Loop through each person in a community
        modularity_dict[name] = i # Create an entry in the dictionary for the person, where the value is which group they belong to.

# Now you can add modularity information like we did the other metrics
nx.set_node_attributes(G, modularity_dict, 'modularity')

## Then in each of these communities, you do your centrality measures like EIGENVECTOR etc etc 

# First get a list of just the nodes in that class
class0 = [n for n in G.nodes() if G.nodes[n]['modularity'] == 0]

# Then create a dictionary of the eigenvector centralities of those nodes
class0_eigenvector = {n:G.nodes[n]['eigenvector'] for n in class0}

# Then sort that dictionary and print the first 5 results
class0_sorted_by_eigenvector = sorted(class0_eigenvector.items(), key=itemgetter(1), reverse=True)

print("Modularity Class 0 Sorted by Eigenvector Centrality:")
for node in class0_sorted_by_eigenvector[:5]:
    print("Name:", node[0], "| Eigenvector Centrality:", node[1])
    
    
### Using eigenvector centrality as a ranking can give you a sense of the important people within this modularity class
##  In smaller networks, it may be a common task to find a list of all the modularity classes and their members

for i,c in enumerate(communities): # Loop through the list of communities
    if len(c) > 2: # Filter out modularity classes with 2 or fewer nodes
        print('Class '+str(i)+':', list(c)) # Print out the classes and their members

### Can ignore this part below, it is just the compiled version of the entire thing from the article

In [91]:
## For each clusterm, perform a network analysis to determine if the learning path of the courses are indeed differnt
# "Quantitative metrics let you differentiate networks, learn about their topologies, and turn a jumble of nodes and edges into something you can learn from."

# Find the shortest path from one content to the other?

# DENSITY
# Density is the ratio of the actual edges in the network to all possible edges in the network
# Interpretation: 0 to 1, the bigger the more dense your network. 1 is a perfectly connect network
# This mean that 
density = nx.density(G)
print("Network density:", density)

# SSSP
# Trim the edge weights first before we try to run SSSP?
# Given some domain knowledge, eg what course the student wants to take, run the SSSP algorithm
# Some thing to consider is that if an intermediate node has a high degree centrality(more on that later),
# The SSSP may take very long since several shortest paths run through him as the mediator
# (This provide us with some hint that DSTA should focus on nodes with the highest degree centrality, 
# I.E make the courses better since these are important courses to take!)
fell_whitehead_path = nx.shortest_path(G, source="Margaret Fell", target="George Whitehead")

print("Shortest path between Fell and Whitehead:", fell_whitehead_path)
print("Length of that path:", len(fell_whitehead_path)-1)

### DIAMETER
# We can also look at dimaeter, which the longest of the shortest path.
# This gives as a sense of the network's overall size. 
# I.e this means that the longest it will take for a student to accomplish this is ____. (provided we used time as edge weights) 
# Should we tell our client to look into the longest short path, and tell them that these are the learning paths that are
# taking the longest to complete?

## If the graph is not connected, then the nx.diameter will throw error
diameter = nx.diameter(subgraph)
print("Network diameter of largest component:", diameter)

# COMPONENTS, AND COMPONENTS AND THIER DIAMETERS
# LOOK AT THE NUMBER OF COMPONENTS
# If we have more than one components, then this means that some course contents are very different from the 
# Eg interpretation: There is a learning path frokm length 8 between the two furthest apart courses in the network.

# If your Graph has more than one component, this will return False:
print(nx.is_connected(G))

# Next, use nx.connected_components to get the list of components,
# then use the max() command to find the largest one:
components = nx.connected_components(G)
largest_component = max(components, key=len)

# Create a "subgraph" of just the largest component
# Then calculate the diameter of the subgraph, just like you did with density.
# This ensures that nx.diameter will not throw an error
subgraph = G.subgraph(largest_component)
diameter = nx.diameter(subgraph)
print("Network diameter of largest component:", diameter)


# Triadic closure: if two people know the same person, they are likdely to know each other
# The number of these enclosed triangles in the network can be used to
# find clusters and communities of courses that are related to each other?

# TRANSITIVITY
# Clustering coefficient is one way to measure Triadic closure as it measures the clustering tendency.
# Transitivity is the ratio of all triangles over all possible triangles
triadic_closure = nx.transitivity(G)
print("Triadic closure:", triadic_closure)


# CENTRALITY
# Answers the questions of which nodes are the most important?
# Degree, betweenness centrality, eigenvector centrality etc etc

## DEGREE
# Course with the highest degree are those most connected to other courses. These nodes are call Hubs.
degree_dict = dict(G.degree(G.nodes()))
nx.set_node_attributes(G, degree_dict, 'degree') # store all the degrees in the node
### Find the top few nodes with the highest degree
#  Eg print(G.nodes['William Penn'])
# sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)
# print("Top 20 nodes by degree:")
# for d in sorted_degree[:20]:
#     print(d)

## EIGENVECTOR CENTRALITY
# Eigenvector centrality useful for understanding which nodes can get information to other nodes quickly.
# Example: if you know a lot of well-connected people, you can spread a message very efficiently.
# So these are courses that are related to many of the important courses, and should be placed into the learning path?


## BETWEENNESS CENTRALITY
# Betweenness centrality looks at all the shortest paths that pass through a particular node. (Thus takes very long to calculate)
# Tells us which nodes are important not because they have lots of connections themselves, but because they stand between groups, 
# Giving the network connectivity and cohesion

betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality
eigenvector_dict = nx.eigenvector_centrality(G) # Run eigenvector centrality

# Assign each to an attribute in your network
nx.set_node_attributes(G, betweenness_dict, 'betweenness')
nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')

sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)

print("Top 20 nodes by betweenness centrality:")
for b in sorted_betweenness[:20]:
    print(b)
    
### You can see that nodes that have high degree also have high betweenness centrality
# What if you want to know learning path have high betweenness centrality and low degree?
# I.e which high betweenness nodes are unexpected?
## If we have high betweenness centrality and low degree, these could mean that the nodes are important nodes connecting
## distant parts of the graph.

#First get the top 20 nodes by betweenness as a list
top_betweenness = sorted_betweenness[:20]

#Then find and print their degree
for tb in top_betweenness: # Loop through top_betweenness
    degree = degree_dict[tb[0]] # Use degree_dict to access a node's degree, see footnote 2
    print("Name:", tb[0], "| Betweenness Centrality:", tb[1], "| Degree:", degree)
    
    
# COMMUNITIES
## Community detection with modularity
## Modularity is a measure of relative density in your network. 
## A community has a high density relative to other nodes within its module, but low density with those outside
## Modularity gives you an overall score of how fractious your network is, and that is a score
# that can be used to partition the network and return the inidividual communities

# Very dense networks are often more difficult to split into sensible partitions.
# Luckily, networks are not all that dense. Our network is only 0.223
# Although there are some built in approahces like 'minimum cut', modularity is not included in networkX
# We just use an addiitonal python module, YAY
communities = community.greedy_modularity_communities(G)
## the method greedy_modularity_communities() tries to determine the number of communities appropriate for the graphs
## Group all nodes into the subsets based on these communities.

modularity_dict = {} # Create a blank dictionary
for i,c in enumerate(communities): # Loop through the list of communities, keeping track of the number for the community
    for name in c: # Loop through each person in a community
        modularity_dict[name] = i # Create an entry in the dictionary for the person, where the value is which group they belong to.

# Now you can add modularity information like we did the other metrics
nx.set_node_attributes(G, modularity_dict, 'modularity')

## Then in each of these communities, you do your centrality measures like EIGENVECTOR etc etc 

# First get a list of just the nodes in that class
class0 = [n for n in G.nodes() if G.nodes[n]['modularity'] == 0]

# Then create a dictionary of the eigenvector centralities of those nodes
class0_eigenvector = {n:G.nodes[n]['eigenvector'] for n in class0}

# Then sort that dictionary and print the first 5 results
class0_sorted_by_eigenvector = sorted(class0_eigenvector.items(), key=itemgetter(1), reverse=True)

print("Modularity Class 0 Sorted by Eigenvector Centrality:")
for node in class0_sorted_by_eigenvector[:5]:
    print("Name:", node[0], "| Eigenvector Centrality:", node[1])
    
    
### Using eigenvector centrality as a ranking can give you a sense of the important people within this modularity class
##  In smaller networks, it may be a common task to find a list of all the modularity classes and their members

for i,c in enumerate(communities): # Loop through the list of communities
    if len(c) > 2: # Filter out modularity classes with 2 or fewer nodes
        print('Class '+str(i)+':', list(c)) # Print out the classes and their members
        
# Notice that we are filtering out any modularity class with two or fewer nodes.

# it’s usually a good idea to get the global modularity score first 
# to determine whether you’ll learn anything by partitioning your network according to modularity. 
# To see the overall modularity score, 
# take the communities you calculated 
# with communities = community.best_partition(G) and run global_modularity = community.modularity(communities, G). 
# Then just print(global_modularity)

Network density: 0.2237843991007143
